In [45]:
import sh
import json
import os
import sys
import re
import io
import pandas as pd
import subprocess

In [2]:
#LUCENE-SOLR
project_folder = "lucene"
output_folder = "out"

repository_folder = "lucene-solr"
repository_link = "https://github.com/apache/lucene-solr.git"
checkout_branch = "trunk"

issues_folders = ["issue_LUCENE", "issue_SOLR"]
issues_tags = ["LUCENE", "SOLR"]

In [18]:
#CAMEL
project_folder = "camel"
output_folder = "out"

repository_folder = "camel"
repository_link = "https://github.com/apache/camel"
checkout_branch = "master"

issues_folders = ["issue_CAMEL"]
issues_tags = ["CAMEL"]

In [5]:
#ZOOKEEPER
project_folder = "zookeeper"
output_folder = "out"

repository_folder = "zookeeper"
repository_link = "https://github.com/apache/zookeeper"
checkout_branch = "trunk"

issues_folders = ["issue_ZOOKEEPER"]
issues_tags = ["ZOOKEEPER"]

In [46]:
#MAVEN
project_folder = "maven"
output_folder = "out"

repository_folder = "maven"
repository_link = "https://github.com/apache/maven"
checkout_branch = "master"

issues_folders = ["issue_MNG"]
issues_tags = ["MNG"]

In [29]:
#MAHOUT
project_folder = "mahout"
output_folder = "out"

repository_folder = "mahout"
repository_link = "https://github.com/apache/mahout"
checkout_branch = "master"

issues_folders = ["issue_MAHOUT"]
issues_tags = ["MAHOUT"]

In [47]:
def print_progress(i, l, step=1):
    if i % step == 0:
            print("\r>> {}/{}".format(i + 1, l), end="")
            sys.stdout.flush()

In [48]:
if not os.path.exists(os.getcwd() + '/' + project_folder + '/' + repository_folder):
    print("Path doesn\'t exists, cloning repo:")
    git = sh.git.bake(_cwd=project_folder)
    git.clone(repository_link)
else:
    print("Repository exists")

Path doesn't exists, cloning repo:


In [49]:
git = sh.git.bake(_cwd=project_folder + '/' + repository_folder)
git.checkout(checkout_branch)
git.status()

On branch master
Your branch is up-to-date with 'origin/master'.

nothing to commit, working directory clean

In [50]:
def extract_bug(file_path):
    if not file.endswith('.json'):
            return None
    else:
        bug_json_string = open(file_path).read()
        bug = json.loads(bug_json_string)
        bug_fields = bug.get('fields')
        
        if bug_fields['issuetype']['name'] != 'Bug':
            return None
   
        if bug_fields['resolution'] == None:
            return None 
                
        if bug_fields['resolution']['name'] != 'Fixed':
            return None
    return bug

In [51]:
#Extracting all the fixed bugs from the bug repo
bugs = dict() #Dictionary of bugs indexed by TAG+ID (e.g. LUCENE-1234)
for fld in issues_folders:
    path = project_folder + '/' + fld
    dir_list = os.listdir(os.getcwd() + '/' + path)
    l = len(dir_list)
    print()
    print("Progress for " + fld + ":")
    for idx, file in enumerate(dir_list):
        print_progress(idx,l,10)
        bug = extract_bug(path + '/' + file)
        if(bug == None):
            continue
        bugs[bug['key'].upper()] = bug

print()
print(len(bugs), "fixed bugs extracted.")


Progress for issue_MNG:
>> 4661/4663
1412 fixed bugs extracted.


In [52]:
commits = []

log = git.log("--reverse", "--pretty=format:%H")
for c_hash in log:
    if(c_hash.endswith('\n')):
        c_hash = c_hash[:-1]
    commits.append(c_hash)
len(commits)

10169

In [53]:
def get_fixed_bugs(commit_hash):
    details = git.show("--name-only","--pretty=format:%B", commit_hash)
    #init = details.split(':')[0].upper()
    f_keys = []
    
    for tag in issues_tags:
        keys = re.findall(tag + "-[0-9]+",str(details))
        if len(keys) != 0:
            for k in keys:
                if bugs.get(k) != None:
                    #Then it fixes a bug
                    f_keys.append(k)
    return f_keys

In [54]:
#Splits the bug fixing ones from the others

fix_commits = [] #List that will be filled with couples (commit, bugs-fixed)
non_fix_commits = [] #List that will be filled only with commit hashes
l = len(commits)
print("Progress:")
for idx, c in enumerate(commits):
    
    print_progress(idx, l, 10)
        
    details = git.show("--name-only","--pretty=format:%B", c)
    files = git.show("--name-only","--pretty=format:",c)
    
    
    f_keys = get_fixed_bugs(c)    
    if len(f_keys) != 0:
        fix_commits.append((c,f_keys))
    else:
        non_fix_commits.append(c)
        
#Some commits cite more than one issue...which lines are the bug fix, if one of the two is only an improvement ?
#HP: we consider all the removed lines as implicated (fix-inducing)

Progress:
>> 10161/10169

In [55]:
print(len(fix_commits)) #Number of bug fixing commits
print(len(non_fix_commits)) #Number of non bug fixing commits
print(len(commits)) #Number of commits
#We can see that on average half of the commits were only solr related, so we discarded them

1000
9169
10169


In [56]:
def get_added_lines(commitHash):
    
    # Get diff of current and previous commit and store this in a file
    with open(project_folder + '_diff_file.txt', 'wb+') as file:
        cmd = ["git " + "-C ./" + project_folder + '/' + repository_folder 
               + '/' + " diff --no-color " + commitHash +'^ ' + commitHash]
        gitDiff = subprocess.Popen(cmd, shell=True, universal_newlines=True, stdout=file)
        gitDiff.wait()
        file.flush()

    # Get diff of the commit
    output_diff = ""
    with open(project_folder + '_diff_file.txt', 'rb+') as file:
        output_diff = file.read()
    
    file.close()
    
    # Cut output up in separate lines
    lines = output_diff.decode("latin1").split('\n')
    
    line_num = dict()
    start_line = 0
    name_of_file = ""
    i = 0
    name_file_set = set()

    # Go through the diff output to determine line number(s) of deleted lines
    for line in lines:
        # Header where the name of the file is located.
        if line[0:4] == "diff":
            name_of_file = line.split(' b/')[-1]


        # New section of changed code find starting line number
        elif line[0:2] == "@@":
            lineNum = re.findall('\-[0-9]+\,',line)
            start_line = int(re.findall('\-[0-9]+',line)[0][1:])
            i = 0

        # Lines that has been changed
        elif (line[0:2] == "- "):
            if line_num.get(name_of_file) == None:
                # create set of lines that have been changed for each file
                line_num[name_of_file] = []
                name_file_set.add(name_of_file)
            # Add lines to the set that have been changed
            line_num[name_of_file].append(start_line + i);
            i += 1


        # Indentaion infront of a line of code that hasn't changed
        elif line[0:2] == "  ":
            i += 1
            
    return line_num

In [57]:
def get_bug_commits(commit_hash, line_num, bug_hash):
    # checkout previous commit
    git.checkout("--force", commit_hash + "^")

    # Go through all the files that have been changed
    for key_file_name in line_num:
        ## Check if file is a java file if not continue with next file
        if(key_file_name.find('.java') == -1):
            continue
        
        # blame file to see when the lines where last changed (on previous commit)
        with open(project_folder + '_blame_file.txt', 'wb+') as file:
            # --root is added to see boundary commits as normal commits
            cmd = ["git " + "-C ./"+ project_folder + '/' + repository_folder + '/' +" blame -l --root " + key_file_name]
            gitBlame = subprocess.Popen(cmd, shell=True, universal_newlines=True, stdout=file)
            gitBlame.wait()
            file.flush()
            
        output_blame = ""
        with open(project_folder + '_blame_file.txt', 'rb+') as file:
            output_blame = file.read()

        # Cut output up in separate lines
        blame_output = output_blame.decode("latin1")        
        blame_output_split = blame_output.split('\n')
                
        # Get hash for all the lines that have been changed.
        for i in line_num[key_file_name]:
            line = blame_output_split[i-1]
            hash_of_line = line.split(' ')[0]
            
            if bug_hash.get(hash_of_line) == None:
                bug_hash[hash_of_line] = set([key_file_name])
            else:
                bug_hash[hash_of_line].add(key_file_name)
            
    return bug_hash

In [58]:
total = len(fix_commits)
bug_commits = dict()

print("Progress:")
for idx, fix_hash in enumerate(fix_commits):
    new_lines = get_added_lines(fix_hash[0])
    bug_commits = get_bug_commits(fix_hash[0], new_lines, bug_commits)

    print_progress(idx, total,1)

Progress:
>> 1000/1000

In [59]:
sh.rm(project_folder + '_blame_file.txt')
sh.rm(project_folder + '_diff_file.txt')

In [60]:
hash_file_list = []
for key, file_name_set in bug_commits.items():
    for file_name in file_name_set:
        hash_file_list.append([key, file_name, 1])

In [61]:
df = pd.DataFrame(hash_file_list, columns=['sha','file', 'implicated'])
df.to_csv(output_folder + '/' +  project_folder + '_implicated_files.csv', index = False)

df


,sha,file,implicated
0,9131de703971744f9dab58734e4b57f3f2d66dd8,maven-core/src/main/java/org/apache/maven/proj...,1
1,88216c6532dbde3022550cc96a6c521afe8e306d,maven-core/src/main/java/org/apache/maven/proj...,1
2,88216c6532dbde3022550cc96a6c521afe8e306d,maven-embedder/src/main/java/org/apache/maven/...,1
3,88216c6532dbde3022550cc96a6c521afe8e306d,maven-core/src/main/java/org/apache/maven/proj...,1
4,4097d7ae0f25be1c07bc86be100cc4789fa738d2,maven-artifact-manager/src/main/java/org/apach...,1
5,f0cb76ca8edc51321a7aff4fbb5ab76280efe2a5,maven-reports/maven-project-info-reports-plugi...,1
6,f0cb76ca8edc51321a7aff4fbb5ab76280efe2a5,maven-plugins/maven-project-info-reports-plugi...,1
7,e9f55ae5f42f0d924ed2f7adb63bc7a5c074972d,maven-artifact-manager/src/main/java/org/apach...,1
8,004c6cbdf32c3c23899c5f6b8ddca1dd85a8d09c,maven-core/src/main/java/org/apache/maven/plug...,1
9,c6a8b20cbae313650b40059f4c55df86a2ecd8de,maven-core/src/main/java/org/apache/maven/life...,1
